## Model Tuning

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import r2_score, check_scoring
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from housing_functions import *
from sklearn.pipeline import Pipeline


%matplotlib inline

In [26]:
# import data
train=pd.read_csv('../datasets/df_dummies_clean.csv')
test=pd.read_csv('../datasets/test_clean.csv')

In [27]:
# Top model is lasso

y=train['saleprice']
X=train[features_7]
# Test Train split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scale variables
pt = PowerTransformer()
pt.fit(X_train)
X_train = pt.transform(X_train)
X_test = pt.transform(X_test)

# Convert back to matrix / Dataframe
pt_y = PowerTransformer()
pt_y.fit(y_train.to_frame()) 
y_train_pt = pt_y.transform(y_train.to_frame())
y_test_pt = pt_y.transform(y_test.to_frame())


lasso = LassoCV(n_alphas=200, cv=5)
lasso_scores = cross_val_score(lasso, X_train, y_train_pt[:, 0], cv=3)

lasso.fit(X_train, y_train_pt)
lasso.score(X_train, y_train_pt)
lasso_test_score = lasso.score(X_test, y_test_pt)
print(f'Lasso was the closest model and returned a test score of {lasso_test_score}.')
print(" ")
pred = lasso.predict(X_test)
pred_reversed = pt_y.inverse_transform(pred.reshape(-1,1))
print(f'The R2 score is: {r2_score(y_test, pred_reversed)}.')
model=lasso

relevant_coefs=coef_df[coef_df[0]!=0]
relevant_coefs.plot.barh(figsize=(15, 47));

/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:199: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:1100: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Lasso was the closest model and returned a test score of 0.90980270169816.
 
The R2 score is: 0.943528666672559.


NameError: name 'coef_df' is not defined

### Checking metrics

In [ ]:
# Will use sklearn.metrics to check metrics and areas for improvement
sklearn.metrics.check_scoring()

### Gridsearch hyperparameters

## Pipeline to check transform and parameters

In [ ]:
features_7 = train.drop(['ms_subclass','saleprice'], axis=1).columns
y=train['saleprice']
X=train[features_7]
# Test Train split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
pipe = Pipeline([
    ('pt' , PowerTransformer()),
    ('sc' , StandardScaler()),# Transformer
    ('ls' , LassoCV()),          # Estimator
])

# Fit pipe
pipe.fit(X_train, y_train)


In [12]:
# Test pipe
pipe.score(X_test, y_test)

0.9185641430482541

In [13]:
pipe.get_params()

{'memory': None,
 'steps': [('pt',
   PowerTransformer(copy=True, method='yeo-johnson', standardize=True)),
  ('sc', StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('ls',
   LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
           max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
           positive=False, precompute='auto', random_state=None,
           selection='cyclic', tol=0.0001, verbose=False))],
 'verbose': False,
 'pt': PowerTransformer(copy=True, method='yeo-johnson', standardize=True),
 'sc': StandardScaler(copy=True, with_mean=True, with_std=True),
 'ls': LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
         max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
         positive=False, precompute='auto', random_state=None,
         selection='cyclic', tol=0.0001, verbose=False),
 'pt__copy': True,
 'pt__method': 'yeo-johnson',
 'pt__standardize': True,
 'sc__copy': True,
 'sc__with_mean

In [52]:
# Define dictionary of parameters.

pipe_params = {'sc__with_mean': [True, False],
             'sc__with_std' : [True, False],
             'ls__n_alphas' : range(1, 5, 2),
             'ls__random_state' : [42]}

# Instantiate our GridSearchCV object.
pipe_gridsearch = GridSearchCV(pipe, 
                                 pipe_params, 
                                 cv=5, 
                                 verbose=2)



In [31]:
pipe.fit(X_train, y_train)

/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 891593810.2543335, tolerance: 645892082.7880586
  tol, rng, random, positive)


Pipeline(memory=None,
         steps=[('pt',
                 PowerTransformer(copy=True, method='yeo-johnson',
                                  standardize=True)),
                ('sc',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ls',
                 LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001,
                         fit_intercept=True, max_iter=1000, n_alphas=100,
                         n_jobs=None, normalize=False, positive=False,
                         precompute='auto', random_state=None,
                         selection='cyclic', tol=0.0001, verbose=False))],
         verbose=False)

In [33]:
# Fit the GridSearchCV object to the data.
pipe_gridsearch.fit(X_train, y_train);

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] ls__n_alphas=1, ls__random_state=42, sc__with_mean=True, sc__with_std=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[CV]  ls__n_alphas=1, ls__random_state=42, sc__with_mean=True, sc__with_std=True, total=   1.1s
[CV] ls__n_alphas=1, ls__random_state=42, sc__with_mean=True, sc__with_std=True 


/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[CV]  ls__n_alphas=1, ls__random_state=42, sc__with_mean=True, sc__with_std=True, total=   1.2s
[CV] ls__n_alphas=1, ls__random_state=42, sc__with_mean=True, sc__with_std=True 


...

/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[CV]  ls__n_alphas=9, ls__random_state=42, sc__with_mean=False, sc__with_std=False, total=   1.0s
[CV] ls__n_alphas=9, ls__random_state=42, sc__with_mean=False, sc__with_std=False 


/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  2.2min finished
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


[CV]  ls__n_alphas=9, ls__random_state=42, sc__with_mean=False, sc__with_std=False, total=   0.9s


/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [34]:
pipe_gridsearch.best_score_

0.9042329223540011

In [35]:
pipe_gridsearch.best_estimator_

Pipeline(memory=None,
         steps=[('pt',
                 PowerTransformer(copy=True, method='yeo-johnson',
                                  standardize=True)),
                ('sc',
                 StandardScaler(copy=True, with_mean=True, with_std=False)),
                ('ls',
                 LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001,
                         fit_intercept=True, max_iter=1000, n_alphas=5,
                         n_jobs=None, normalize=False, positive=False,
                         precompute='auto', random_state=42, selection='cyclic',
                         tol=0.0001, verbose=False))],
         verbose=False)

In [37]:
pipe_final = pipe_gridsearch.best_estimator_

In [38]:
pipe_final.score(X_test, y_test)

0.919083834978888

### Function for saving predictions as csv

In [48]:
# # Scale variables and Prep Test data and Submission
def submit_score(model, df, features, target, model_number):
    X_submit = df[features]
    X_submit = pt.transform(X_submit) 
    pred = model.predict(X_submit)
   # pred_reversed = pt_y.inverse_transform(pred.reshape(-1,1))
    submit = pd.DataFrame()
    submit['Id'] = df.loc[:,'id']
    submit['SalePrice'] = pred
    filename='submission_'+str(model_number)
    submit.to_csv(f'../Submissions/{filename}.csv',index=False)

In [49]:
submit_score(pipe_final, test, features_7, 'saleprice', 10)

In [68]:
l_alphas = np.logspace(-3, 0, 100)
r_alphas = np.logspace(0,5,100)

pipe_params = {'sc__with_mean': [True, False],
             'sc__with_std' : [True, False],
             'ls__random_state' : [42],
             'ls__alphas' : [r_alphas]}

# Instantiate our GridSearchCV object.
pipe_gridsearch_2 = GridSearchCV(pipe, 
                                 pipe_params, 
                                 cv=5, 
                                 verbose=2)

In [69]:
pipe_gridsearch_2.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] ls__alphas=[1.00000000e+00 1.12332403e+00 1.26185688e+00 1.41747416e+00
 1.59228279e+00 1.78864953e+00 2.00923300e+00 2.25701972e+00
 2.53536449e+00 2.84803587e+00 3.19926714e+00 3.59381366e+00
 4.03701726e+00 4.53487851e+00 5.09413801e+00 5.72236766e+00
 6.42807312e+00 7.22080902e+00 8.11130831e+00 9.11162756e+00
 1.02353102e+01 1.14975700e+01 1.29154967e+01 1.45082878e+01
 1.62975083e+01 1.83073828e+01 2.05651231e+01 2.31012970e+01
 2.59502421e+01 2.91505306e+01 3.27454916e+01 3.67837977e+01
 4.13201240e+01 4.64158883e+01 5.21400829e+01 5.85702082e+01
 6.57933225e+01 7.39072203e+01 8.30217568e+01 9.32603347e+01
 1.04761575e+02 1.17681195e+02 1.32194115e+02 1.48496826e+02
 1.66810054e+02 1.87381742e+02 2.10490414e+02 2.36448941e+02
 2.65608778e+02 2.98364724e+02 3.35160265e+02 3.76493581e+02
 4.22924287e+02 4.75081016e+02 5.33669923e+02 5.99484250e+02
 6.73415066e+02 7.56463328e+02 8.49753436e+02 9.54548457e+02
 1.072267

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2863: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 655881000.3881836, tolerance: 524273492.1811086
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did no

/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 94968831222.70795, tolerance: 524273492.1811086
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 95857165871.04185, tolerance: 524273492.1811086
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 96716972336.48561, tolerance: 524273492.1811086
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_

/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1018894034.7552795, tolerance: 505311564.80376136
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9983446746.293396, tolerance: 505311564.80376136
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20866667297.514557, tolerance: 505311564.80376136
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordi

[CV]  ls__alphas=[1.00000000e+00 1.12332403e+00 1.26185688e+00 1.41747416e+00
 1.59228279e+00 1.78864953e+00 2.00923300e+00 2.25701972e+00
 2.53536449e+00 2.84803587e+00 3.19926714e+00 3.59381366e+00
 4.03701726e+00 4.53487851e+00 5.09413801e+00 5.72236766e+00
 6.42807312e+00 7.22080902e+00 8.11130831e+00 9.11162756e+00
 1.02353102e+01 1.14975700e+01 1.29154967e+01 1.45082878e+01
 1.62975083e+01 1.83073828e+01 2.05651231e+01 2.31012970e+01
 2.59502421e+01 2.91505306e+01 3.27454916e+01 3.67837977e+01
 4.13201240e+01 4.64158883e+01 5.21400829e+01 5.85702082e+01
 6.57933225e+01 7.39072203e+01 8.30217568e+01 9.32603347e+01
 1.04761575e+02 1.17681195e+02 1.32194115e+02 1.48496826e+02
 1.66810054e+02 1.87381742e+02 2.10490414e+02 2.36448941e+02
 2.65608778e+02 2.98364724e+02 3.35160265e+02 3.76493581e+02
 4.22924287e+02 4.75081016e+02 5.33669923e+02 5.99484250e+02
 6.73415066e+02 7.56463328e+02 8.49753436e+02 9.54548457e+02
 1.07226722e+03 1.20450354e+03 1.35304777e+03 1.51991108e+03
 1.7073

...

[CV]  ls__alphas=[1.00000000e+00 1.12332403e+00 1.26185688e+00 1.41747416e+00
 1.59228279e+00 1.78864953e+00 2.00923300e+00 2.25701972e+00
 2.53536449e+00 2.84803587e+00 3.19926714e+00 3.59381366e+00
 4.03701726e+00 4.53487851e+00 5.09413801e+00 5.72236766e+00
 6.42807312e+00 7.22080902e+00 8.11130831e+00 9.11162756e+00
 1.02353102e+01 1.14975700e+01 1.29154967e+01 1.45082878e+01
 1.62975083e+01 1.83073828e+01 2.05651231e+01 2.31012970e+01
 2.59502421e+01 2.91505306e+01 3.27454916e+01 3.67837977e+01
 4.13201240e+01 4.64158883e+01 5.21400829e+01 5.85702082e+01
 6.57933225e+01 7.39072203e+01 8.30217568e+01 9.32603347e+01
 1.04761575e+02 1.17681195e+02 1.32194115e+02 1.48496826e+02
 1.66810054e+02 1.87381742e+02 2.10490414e+02 2.36448941e+02
 2.65608778e+02 2.98364724e+02 3.35160265e+02 3.76493581e+02
 4.22924287e+02 4.75081016e+02 5.33669923e+02 5.99484250e+02
 6.73415066e+02 7.56463328e+02 8.49753436e+02 9.54548457e+02
 1.07226722e+03 1.20450354e+03 1.35304777e+03 1.51991108e+03
 1.7073

/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 666136679.4587402, tolerance: 645892082.7880586
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 859757746.9324341, tolerance: 645892082.7880586
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWar

/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11725464887.22815, tolerance: 643395564.8487977
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28743618075.26062, tolerance: 643395564.8487977
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 45305163623.155914, tolerance: 643395564.8487977
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate

/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 146788119137.79633, tolerance: 643395564.8487977
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3456565941.6867065, tolerance: 637076328.2327474
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16828507756.081238, tolerance: 637076328.2327474
  tol, rng, random, positive)
/Users/allanshomefolder/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordina

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('pt',
                                        PowerTransformer(copy=True,
                                                         method='yeo-johnson',
                                                         standardize=True)),
                                       ('sc',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('ls',
                                        LassoCV(alphas=None, copy_X=True,
                                                cv='warn', eps=0.001,
                                                fit_intercept=True,
                                                max_iter=1000, n_alphas=100,
                                           

In [72]:
print(f'Best Score is {pipe_gridsearch_2.best_score_}')
print(f'Best Estimator is {pipe_gridsearch_2.best_estimator_}')
print(f'Model achieved a test score of {pipe_gridsearch_2.score(X_test, y_test)}')

Best Score is 0.9038013601774101
Best Estimator is Pipeline(memory=None,
         steps=[('pt',
                 PowerTransformer(copy=True, method='yeo-johnson',
                                  standardize=True)),
                ('sc',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ls',
                 LassoCV(alphas=array([1.00000000e+00, 1.12332403e+00, 1.26185688e+00, 1.41747416e+00,
       1.59228279e+00, 1.78864953e+00, 2.00923300e+00, 2.25701972e+00,
       2.53536449e+00, 2.84803587e+00, 3.1992671...
       2.78255940e+04, 3.12571585e+04, 3.51119173e+04, 3.94420606e+04,
       4.43062146e+04, 4.97702356e+04, 5.59081018e+04, 6.28029144e+04,
       7.05480231e+04, 7.92482898e+04, 8.90215085e+04, 1.00000000e+05]),
                         copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
                         max_iter=1000, n_alphas=100, n_jobs=None,
                         normalize=False, positive=False, precompute='aut

In [74]:
submit_score(pipe_gridsearch_2, test, features_7, y, 11)